In [1]:
import os
import sys
import pandas as pd


%pwd
os.chdir("../")

# Add the absolute path to src/ so Python can find automatch
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)
    
%pwd


'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [2]:
%pwd

'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [3]:
from autoMatch.utils.snowflake_utils import get_snowpark_session
session = get_snowpark_session()

[2025-10-02 15:56:23,723: INFO: connection: Snowflake Connector for Python Version: 3.7.0, Python Version: 3.10.11, Platform: Windows-10-10.0.26100-SP0]
[2025-10-02 15:56:23,725: INFO: connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.]
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/e2ba81b8-03fe-407c-96a1-f4bc0f512e7d/saml2?SAMLRequest=nZJBc9owEIX%2Fikc925KNAaMBMk4oDTNpSoBkmt6EvCYqtuRKMg799ZUNzCSH5NCbRnq739O%2BHV%2B9loV3AG2EkhMUBgR5ILnKhNxN0ONm7ifIM5bJjBVKwgQdwaCr6diwsqhoWtsXuYI%2FNRjruUbS0PZhgmotqWJGGCpZCYZaTtfp9zsaBYQyY0Bbh0PnkswIx3qxtqIYN00TNL1A6R2OCCGYj

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: str
    database: str
    schema: str
    input_table: str
    columns: dict
    status_file: str


In [ ]:
from autoMatch.constants import *
from autoMatch.utils.common import read_yaml, create_directories
from autoMatch import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            database=config.database,
            schema=config.schema,
            input_table=config.input_table,
            columns = schema.columns,
            status_file = config.STATUS_FILE
        )

        return data_validation_config

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self, session)-> bool:
        try:
            validation_status = True

            database = self.config.database
            schema = self.config.schema
            input_table = self.config.input_table
            columns = self.config.columns
            status_file = self.config.status_file


            df = session.table(f"{database}.{schema}.{input_table}")

            df_cols = [col.lower() for col in df.columns]
            schema_cols = [col.lower() for col in columns.keys()]

            missing_columns = list(set(schema_cols) - set(df_cols))
            extra_columns = list(set(df_cols) - set(schema_cols))

            for col in missing_columns:
                validation_status = False
                logger.info(f"Column {col} from schema is missing in the dataframe")

            for col in extra_columns:
                logger.info(f"Column {col} is present in the dataframe but not specified in the schema")

            for field in df.schema.fields:
                #print(f"Column: {field.name}, Type: {field.datatype}")
                for col_name, col_type in columns.items():
                    if col_name == field.name.lower():
                        #print(f"Comparing {col_name} ({col_type}) with {field}:")
                        if col_type not in str(field.datatype).lower():
                            validation_status = False
                            logger.info(f"Dataframe column {col_name} with type ({col_type}) does not match schema column type {field.name} ({field.datatype})")

            with open(status_file, 'w') as f:
                f.write(f"Validation status: {validation_status}")



            return validation_status
        
        except Exception as e:
            raise e

In [41]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    df = data_validation.validate_all_columns(session)
except Exception as e:
    raise e

[2025-10-02 16:49:40,546: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-02 16:49:40,548: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-02 16:49:40,552: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-02 16:49:40,554: INFO: common: created directory at: artifacts]
[2025-10-02 16:49:40,555: INFO: common: created directory at: artifacts/data_validation]
[2025-10-02 16:49:40,557: INFO: cursor: query: [SELECT  *  FROM (IT_DISCOVERY.CONSUMER_INT_MODEL.MPG_IT_AUTOMATCH_CANDIDATE)]]
[2025-10-02 16:49:40,638: INFO: cursor: query execution done]
[2025-10-02 16:49:40,640: INFO: cursor: Number of results in first chunk: 0]
Column: DATE_ADDED, Type: TimestampType(tz=ntz)
Comparing date_added (timestamp) with StructField('DATE_ADDED', TimestampType(tz=ntz), nullable=True):
Column: CANDIDATEID, Type: LongType()
Comparing candidateid (long) with StructField('CANDIDATEID', LongType(), nullable=True):
Column: DESCRIPTION, Type: StringTy